# Radiative Processes Project: Binary Disk Spectrum
My proposed Jupyter notebook project for the Radiative Processes class is a toy-model python script to calculate the spectrum from a gas disk around an unequal-mass, accreting, massive black hole binary (MBHB). MBHBs are predicted to form from galaxy mergers after the respective black holes within the two galaxies sink towards the center due to dynamical friction. Unequal mass MBHBs are more likely to form than equal mass systems. Gas remnants from the galaxy merger will settle into 3 disks around the MBHB system: an inner disk around the primary (or larger) black hole, a secondary mini-disk around the secondary (or smaller) black hole, and the outer disk that extends from the secondary black hole's orbit outwards. Assuming a steady state disk, we can estimate the spectrum of each disk to be a blackbody spectrum. My code will take initial conditions (such as mass of the binary, mass ratio of black holes, and Eddington accretion rate factor) and output the total spectrum of the MBHB system by adding the blackbody spectrum for each disk. This is directly relevant to my research and has never been produced in the literature. Later, I will extend this project to take initial conditions from my simulations of MBHB systems inspiralling towards merger to determine how the spectral appearance of MBHB evolves over the inspiral.

The next cell will import the necessary python libraries

In [1]:
from numpy import logspace, pi, sqrt, exp, zeros_like, log10, diff, log
from matplotlib import pyplot as plt 
import matplotlib as mpl 
mpl.rcParams.update(mpl.rcParamsDefault) 
import sys 
from ipywidgets import interact, interactive 
import ipywidgets as widgets 
!{sys.executable} -m pip install --quiet matplotlib

Here we define the constants for any MBHB system in cgs units:

In [2]:
G = 6.67e-8  # cm^3 g^-1 s^-2 
Msun = 2e33  # g 
sb = 5.67e-5  # g s^-3 K^-4: Stefan Boltzman Constant in cgs 
c = 3e10  # cm s^-1 
h = 6.626e-27  # ergs s: Planck Constant in cgs 
k = 1.38e-16  # ergs K^-1 
year = 3.15e7 # s 
tsal = 5e7 * year # s: salpeter time 
erg = 6.242e11  # eV

Below are functions for the accretion rate, innermost stable circular orbit (ISCO) around a black hole and the Hill radius of the secondary black hole. 
The Accretion rate is defined as a fraction of the Eddington accretion rate. 
The radius of the innermost stable circular orbit is found assuming a Schwarzschild spacetime, i.e. non-spinning black hole. 
The Hill radius is found given the mass ratio of the binary; this is Eqn. 2 of Eggleton (1982).

In [1]:
def Mdot_acc(M, edd_frac): 
  return edd_frac * M / tsal 
 
def Risco(M): 
  return 12 * G * M / c**2 
 
def HillR(q): 
  return 0.49 * (q ** (2 / 3)) / (0.6 * q ** (2 / 3) + log(1 + q ** (1 / 3)))

Next we can define the temperature of a disk around a black hole as well as the Planck spectrum for a black hole disk: 
The surface temperature of a steady-state disk with inner radius R0 is defined first; this is Eqn. 3.23 of Pringle (1981). 
The temperature of the inner edge of a steady-state accretion disk is defined; this equation appears below Eqn. 3.23 of Pringle (1981), note there is a typo in the paper: the R is missing a subscript 'star'. 
The specific intensity of a blackbody of temperature T, and frequency nu is found using the Planck function.

In [1]:
def disk_temperature(R: float, R0: float, M: float, Mdot: float): 
   return Tstar(R0, M, Mdot) * ((R0 / R) ** 3 * (1 - sqrt(R0 / R))) ** 0.25 
 
def Tstar(R0: float, M: float, Mdot: float): 
  return (3 * G * M * Mdot / (8 * pi * R0**3 * sb)) ** 0.25 
 
def planck_spectrum(T, nu): 
  return 2 * h * nu**3 / c**2 / (exp(h * nu / (k * T)) - 1)

Finally, here is a function for the disk spectrum using the functions defined above: 
This function produces a disk spectrum. It uses a 1st-order integral over disk radius to produce the multi-temperature Blackbody spectrum of a disk, which extends from R0 to Rout. Note: the frequency range is currently hard-coded. Note: multiply the result by pi to obtain the isotropic specific power. 

In [1]:
def disk_spectrum(M: float, Mdot: float, R0: float, Rout: float, delta: float = 0.1): 
  nu_v = logspace(12, 19, 999) 
  nu_c = 0.5 * (nu_v[1:] + nu_v[:-1]) 
  R = R0 * 1.0001 
  R1 = Rout 
  Inu = zeros_like(nu_c) 
 
  if Rout < R0: 
     print('Warning: Rout < R1, the spectrum is zero') 
 
  while R < Rout: 
     dR = R * delta 
     Inu += 2 * pi * R * dR * planck_spectrum(disk_temperature(R, R0, M, Mdot), nu_c) 
     R *= 1.0 + delta 
  return nu_v, Inu

Below is a function that plots the spectra for a given binary mass, mass ratio, preferrential accretion, and number of orbits until merger. (Inu1 is the primary disk spectrum, Inu2 is the secondary disk spectrum, Inu3 is the outer disk spectrum)

In [1]:
def plot_spectrum(M, q, pref_acc, Notm): 
  M = 1e7 * M # Mass of the binary in grams 
  M1 = M / (1 + q) # Mass of the primary black hole in g 
  M2 = M1 * q # Mass of the secondary black hole in g 
  eta = q / (1 + q) ** 2 # Constant that depends on mass ratio 
  edd_frac_1 = (1 + q) / (1 + pref_acc) # Factor of Eddington acretion onto the primary black hole 
  edd_frac_2 = pref_acc * edd_frac_1 / q # Factor of Eddington acretion onto the secondary black hole 
  Mdot1 = Mdot_acc(M1, edd_frac_1) # Accretion rate of the primary black hole in g / s 
  Mdot2 = Mdot_acc(M2, edd_frac_2) # Accretion rate of the secondary black hole in g / s 
  Mdot = Mdot_acc(M, 1.0) # Accretion rate of the binary in g / s 
  a0 = G * M / c**2 * (5 / (128 * pi * eta * 4 * Notm)) ** (-2 / 5) # Initial separation in cm 
 
  nu_v, Inu1 = disk_spectrum(M1, Mdot1, Risco(M1), a0 - HillR(q) * a0, delta=0.001) 
  nu_v, Inu2 = disk_spectrum(M2, Mdot2, Risco(M2), HillR(q) * a0, delta=0.001) 
  nu_v, Inu3 = disk_spectrum(M, Mdot, a0 + HillR(q) * a0, 1e6 * a0, delta=0.001) 
  nu = 0.5 * (nu_v[1:] + nu_v[:-1]) 
  Lnu1 = pi * Inu1 * nu 
  Lnu2 = pi * Inu2 * nu 
  Lnu3 = pi * Inu3 * nu 
  Lnutot = Lnu1 + Lnu2 + Lnu3 # Isotropic Specific Power 
 
  fig = plt.figure() 
  ax = fig.add_subplot(111) 
  colors = {'blue': '#1f77b4', 'green': '#2ca02c','red': '#d62728'} 
 
  ax.plot(h * nu * erg, Lnu1, color=colors['green'], label='Inner Disk', linewidth=1) 
  ax.plot(h * nu * erg, Lnu2, color=colors['blue'], label='Secondary Disk', linewidth=1) 
  ax.plot(h * nu * erg, Lnu3, color=colors['red'], label='Outer Disk', linewidth=1) 
  ax.plot(h * nu * erg, Lnutot, '--', color='k', label='Total', linewidth=2) 
 
  ax.set_xlabel(r'$h \nu \rm{[eV]}$') 
  ax.set_ylabel(r'$\nu L_{\nu} \rm{[erg / s]}$') 
  ax.set_xscale('log') 
  ax.set_yscale('log') 
  ax.set_ylim(5e40, 2e44) 
  ax.set_xlim(0.1, 4000) 
  ax.legend(loc='lower left') 
  fig.tight_layout(pad=0.1) 
  plt.show()

Finally, we will plot the spectra of the MBHB system with widgets to change the mass, mass ratio, preferrential accretion, and number of orbits until merger:

In [1]:
interact(plot_spectrum, M = widgets.BoundedFloatText(value=1e7, min=1e4, max = 1e8, step=1e4, descriptions='Binary Mass [Msun]:', disabled=False), q = widgets.BoundedFloatText(value=0.01, min=0.001, max = 1.0, step=0.001, descriptions='Mass ratio:', disabled=False), pref_acc = widgets.BoundedFloatText(value=10, min=1, max = 10, step=0.1, descriptions='Preferrential Accretion:', disabled=False), Notm = widgets.BoundedFloatText(value=1e4, min=1, max = 1e4, step=1, descriptions='# orbits till merger:', disabled=False)) 